##### A general approach of identifying and verifying people from photographs of their face.
"FaceNet, that directly learns a mapping from face images to a compact Euclidean space where distances directly correspond to a measure of face similarity."

In [13]:
# Loading the keras facenet model
from keras.models import load_model

In [14]:
# load the model
model = load_model('./model/facenet_keras.h5')

#### Detect Faces for Face Recognition

In [24]:
from pillow import Image      # Python Imaging Library
from numpy import asarray  # undamental package for scientific computing
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import load
from numpy import expand_dims
from os import listdir
from os.path import isdir

ModuleNotFoundError: No module named 'pillow'

In [25]:
# Importing the Multi-Task Cascaded Convolutional Neural Network, or MTCNN, for face detection,
#              e.g. finding and extracting faces from photos
from mtcnn.mtcnn import MTCNN
# mtcnn library is to create a face detector and extract faces for our use with the FaceNet face detector models 

# print version
print(mtcnn.__version__)

ModuleNotFoundError: No module named 'cv2'

facenet_keras.h5 model is the pre-trained Keras FaceNet model provided by Hiroki Taniai. 
- It was trained on MS-Celeb-1M dataset and expects input images to be color, to have their pixel values whitened (standardized across all three channels), and to have a square shape of 160×160 pixels.

In [4]:
# summarize input and output shape
print(model.inputs)
print(model.outputs)

[<tf.Tensor 'input_1:0' shape=(?, 160, 160, 3) dtype=float32>]
[<tf.Tensor 'Bottleneck_BatchNorm/cond/Merge:0' shape=(?, 128) dtype=float32>]


In [26]:
def extract_face( filename ):
    # Load an image as a NumPy array from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = asarray(image)

    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # The result is a list of bounding boxes, where each bounding box defines a lower-left-corner of the bounding box,
    #      as well as the width and height.

    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    # bug fix
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize( (160, 160) ) # the model expects the imagewith size 160 & 160
    face_array = asarray(image)
    return face_array

In [27]:
# The load_faces() function below will load all of the faces for all images in a directory
def load_faces(directory):
    faces = list()
    # enumerate files
    for filename in listdir(directory):
        # path
        path = directory + filename
        # get face
        face = extract_face(path)
        # store
        faces.append(face)
    return faces

In [28]:
# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
    X, y = list(), list()
    # enumerate folders, on per class
    for subdir in listdir(directory):
        # path
        path = directory + subdir + '/'
        # skip any files that might be in the dir
        if not isdir(path):
            continue
        # load all faces in the subdirectory
        faces = load_faces(path)
        # create labels
        labels = [subdir for _ in range(len(faces))]
        # summarize progress
        print('>loaded %d examples for class: %s' % (len(faces), subdir))
        # store
        X.extend(faces)
        y.extend(labels)
    return asarray(X), asarray(y)

In [29]:
# get the face embedding for one face
def get_embedding(model, face_pixels):
	# scale pixel values
	face_pixels = face_pixels.astype('float32')
	# standardize pixel values across channels (global)
	mean, std = face_pixels.mean(), face_pixels.std()
	face_pixels = (face_pixels - mean) / std
	# transform face into one sample
	samples = expand_dims(face_pixels, axis=0)
	# make prediction to get embedding
	yhat = model.predict(samples)
	return yhat[0]

In [30]:
# load train dataset
trainX, trainy = load_dataset('./Data/train/')
print(trainX.shape, trainy.shape)
# load test dataset
testX, testy = load_dataset('./Data/val/')
# save arrays to one file in compressed format
savez_compressed('5-celebrity-faces-dataset.npz', trainX, trainy, testX, testy)

NameError: name 'listdir' is not defined

In [31]:
# load the face dataset
data = load('5-celebrity-faces-dataset.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)

NameError: name 'load' is not defined

In [ ]:
# convert each face in the train set to an embedding
newTrainX = list()
for face_pixels in trainX:
    embedding = get_embedding(model, face_pixels)
    newTrainX.append(embedding)
newTrainX = asarray(newTrainX)
print(newTrainX.shape)

In [ ]:
# convert each face in the test set to an embedding
newTestX = list()
for face_pixels in testX:
    embedding = get_embedding(model, face_pixels)
    newTestX.append(embedding)
newTestX = asarray(newTestX)
print(newTestX.shape)

In [ ]:
# save arrays to one file in compressed format
savez_compressed('5-celebrity-faces-embeddings.npz', newTrainX, trainy, newTestX, testy)

References
1. FaceNet: A Unified Embedding for Face Recognition and Clustering https://arxiv.org/abs/1503.03832
2. Handbook of Face Recognition book ( TODO need to read )
3. https://machinelearningmastery.com/how-to-develop-a-face-recognition-system-using-facenet-in-keras-and-an-svm-classifier/
